In [1]:
import os
from glob import glob
import sys
import pandas as pd
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader

sys.path.insert(0, '../')
from dataset import REDataset, split_train_test_loader, load_data
from models import load_model
from tokenization import load_tokenizer
from config import Config, ModelType, PreTrainedType, TokenizationType

from tqdm import tqdm
import numpy as np

In [2]:
device = Config.Device
model = load_model()

Load Model...	Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized fro

In [3]:
model.to(device)
print(device)

cuda:0


In [9]:
dataset = REDataset()
train_loader, valid_loader = split_train_test_loader(dataset)

Load Tokenizer...	done!
Load raw data...	done!
Apply Tokenization...	done!


In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.005)

In [11]:
model.train()
print('train')

train


In [12]:
# ACC, RECALL, PRECISION, F1
pred_list = []
true_list = []

# CE Loss
total_loss = 0
num_samples = 0

for sentences, labels in tqdm(train_loader, desc='Train'):
    outputs = model(**sentences).logits
    loss = criterion(outputs, labels)
    total_loss += loss.item()
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    num_samples += len(labels)
    
    _, preds = torch.max(outputs, dim=1)
    preds = preds.data.cpu().numpy()
    labels = labels.data.cpu().numpy()

    pred_list.append(preds)
    true_list.append(labels)

train_loss = total_loss / num_samples
precision, recall, f1, _ = precision_recall_fscore_support(y_true=true_arr, y_pred=pred_arr, average='macro')


Train: 100%|██████████| 225/225 [00:54<00:00,  4.09it/s]


In [30]:
true_arr = np.hstack(true_list)
pred_arr = np.hstack(pred_list)

Macro 통일

In [34]:
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, precision_recall_fscore_support

In [42]:
precision, recall, f1, _ = precision_recall_fscore_support(y_true=true_arr, y_pred=pred_arr, average='macro')


/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(0.022919117099058264, 0.024378546196001, 0.019240300806075167, None)

In [18]:
def foo():
    accuracy_score(y_true=true_np, y_pred=pred_np)

%timeit foo()


474 µs ± 659 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [59]:
loss

tensor(3.3002, device='cuda:0', grad_fn=<NllLossBackward>)

In [60]:
loss.backward()
loss

tensor(3.3002, device='cuda:0', grad_fn=<NllLossBackward>)

In [34]:
labels.detach().cpu().numpy()

array([ 0,  0,  0,  1,  0,  0, 22, 25,  6,  7,  0, 10, 33,  0,  0,  5,  5,
        0,  4, 21,  2,  3,  0, 21,  0,  0,  0,  4,  0,  0,  0,  4])

array([16, 16, 27,  0, 16,  3,  3,  0,  7, 31, 40,  3, 16, 27,  3,  3, 16,
        3,  0,  7,  3, 16, 16, 11, 40,  0, 33, 16, 11, 31, 33,  3])

In [22]:
criterion(output, label)

tensor(3.6168, device='cuda:0', grad_fn=<NllLossBackward>)

In [14]:
output.logits

tensor([[ 0.2404,  0.1699, -0.2687,  ..., -0.1668,  0.2764,  0.2006],
        [ 0.3447, -0.1557, -0.1958,  ..., -0.2912,  0.1015,  0.1041],
        [ 0.1219,  0.0252, -0.1080,  ..., -0.1125,  0.2804,  0.2249],
        ...,
        [ 0.3234, -0.0316, -0.1043,  ..., -0.1559,  0.2202,  0.1416],
        [ 0.5335, -0.2708, -0.1581,  ...,  0.0153,  0.1181,  0.1537],
        [ 0.2040, -0.0679, -0.0913,  ..., -0.0946,  0.1621,  0.1360]],
       device='cuda:0', grad_fn=<AddmmBackward>)